<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span><ul class="toc-item"><li><span><a href="#Issues-with-Cloneable-interface" data-toc-modified-id="Issues-with-Cloneable-interface-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Issues with Cloneable interface</a></span></li><li><span><a href="#Alternatives-to-Cloneable-interface" data-toc-modified-id="Alternatives-to-Cloneable-interface-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Alternatives to Cloneable interface</a></span></li></ul></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Example</a></span></li></ul></div>

# Prototype

## Intro

This pattern refers to creating a duplicate object while keeping performance in mind. We specify the type of object to create with a prototypical instance. The new instance copies the prototype completely.

This is used when creation of an object is costly. This means that the focus is to minimise the cost of the creation of the new instance.

It provides a mechanism to copy the original object and then modify it to fit our needs. In Java, we use cloning (Shallow) to achieve this or deserialisation when we need a complete copy including nested structures and linked objects.

This pattern mandates that the object itself handles the production of the new instance.

A good example of a real world example is a template doc that is copied by a user, then changed. Changing the template will change the consequent copies of said template.

The best place to use this pattern is when we need to instantiate classes at runtime or when we want to map complex hierarchies across factories to actual objects.

Advantages include:
* Hides complexities of making new objects
* Provides the option for client to generate objects whose type is not known until runtime
* Can be more efficient than creating a new object via the constructor

Disadvantages are:
* Any subclasses have to implement the cloning mechanism too
    * This can be very challenging due to objects potentially not supporting it
* The cloneable interface needed for this behaviour (at least for shallow copies) has some issues

### Issues with Cloneable interface

This is a demic issue (not really a real world concern). However, there are some points that the demics have on this.

The cloneable interface lacks the clone() method, meaning we **have** to remember to implement this ourselves on both the inheriting class and any subclasses. Not a major issue, but still a concern.

Object.clone() is protected. This means that any object already has a clone method that is protected. To implement the simple approach to the prototype pattern, we need to indirectly call this in our own implementation. This of course is very messy and gets confusing quick.

Based on the above 2 points, Object.clone() does not invoke the constructor of a cloned object, meaning there is no guarantee that the object cloned is even going to be a complete prototype.

Object.clone() only supports shallow copy. So, any relationships and nested structures are not copied. Arrays can be copied simply as part of the pattern. But, class refs cannot easily and require more code.

Any classes related to the prototype (up and downstream) need to implement the clone method (as it isn't part of the interface, subclasses don't have access once implemented on a parent). This adds, potentially, loads of code.

Final fields on objects cannot be manipulated as they can only be changed by constructors which Object.clone() cannot invoke. There are ways around this (class variables that contain a collection of all created instances), but this is jank.

### Alternatives to Cloneable interface

In [10]:
public class Employee {
    public int id;
    public String name;
    
    public Employee(Employee employee) {
        this.id = employee.id;
        this.name = employee.name;
    }
}

The above is a copy constructor. It takes in an instance of the class itself and returns a copy. This resolves many of the Object.clone() issues and is much more readable. But, there is still and issue with any subclasses from a prototype.

In [11]:
public class Manager extends Employee {

    public Manager(Manager manager) {
        super(manager.id, manager.name);
    }
}

CompilationException: 

The error here is partially a symptom of the Jupyter Notebook. But, the object in question being passed in would still give a similar error due to not being of the correct type. We could cast to get around this (we cast anyway using the cloneable interface), but this could still cause a ClassCastException.

To get arround this, we can reimplement the parent class like:

In [25]:
public class Employee {

    public int id;
    public String name;

    public Employee(int id, String name) {
        this.id = id;
        this.name = name;
    }

    public Employee(Employee employee) {
        this.id = employee.id;
        this.name = employee.name;
    }

    public Employee copy() {
        return new Employee(this);
    }
}

This will mean that the class returned in **any** situation will always be of the correct type and because we are referencing the constructors in the respective clone method, any relationships created there will be created too. This is the most common way across languages to implement the prototype pattern.

In [27]:
Employee emp = new Employee(1, "Edward");

In [28]:
Employee emp2 = emp.copy();

In [29]:
System.out.println(emp.name);
System.out.println(emp2.name);

Edward
Edward


Now, to implement the subclass:

In [32]:
public class Manager extends Employee {

    public Manager(int id, String name) {
        super(id, name);
    }

    public Manager(Manager manager) {
        super(manager.id, manager.name);
    }

    @Override
    public Employee copy() {
        return new Manager(this);
    }
}

Note that we override the parent class' copy method here, but it has an identical signature. Because of Java's baked in polymorphism, we have to do this. Luckily, Java enforces Liskov sub, therefore meaning child classes can replace parents.

## Implementation

In [43]:
// Prototype

abstract class Shape implements Cloneable {
    protected String id;
    protected String type;
    
    public String getType(){
        return this.type;
    }
    
    public Object clone(){
        Object clone = null;
        
        try {
            clone = super.clone();
        } catch(CloneNotSupportedException exception){
            exception.printStackTrace();
        }
        
        return clone;
    }
}

In [44]:
class Rectangle extends Shape {
    public Rectangle(){
        id = "1";
        type = "Rectangle";
    }
}

There is no need to implement the clone method again here on the subclass as there are no new variables introduced.

In [45]:
class ShapeCache {
    private static Hashtable<String, Shape> shapeMap = 
                                                new Hashtable<String, Shape>();
    
    public static Shape getShapeFromHash(String shapeId){
        Shape cachedShape = shapeMap.get(shapeId);
        return (Shape) cachedShape.clone();
    }
    
    public static void loadCache(){
        Rectangle rect = new Rectangle();
        shapeMap.put(rect.id, rect);
    }
}

In [46]:
ShapeCache.loadCache();

In [52]:
Shape myShape = ShapeCache.getShapeFromHash("1");
Shape myShape2 = ShapeCache.getShapeFromHash("1");

In [54]:
System.out.println(myShape);
System.out.println(myShape2);

System.out.println(myShape.id);
System.out.println(myShape2.id);

REPL.$JShell$28B$Rectangle@40c826e7
REPL.$JShell$28B$Rectangle@d2def5e4
1
1


The static helper cache was bought into create objs for us so we weren't calling a constructor. The objects live in the hashtable and are copied out using the clone method. Note how we have to cast the object before returning. This is because the return is always an object and not the type actually stored. This is a limitation of clone().

Check the copy constructor above for the best way to implement this pattern. This way is the best technically, but is very overhead heavy and rigid with how clients can use objects. Could be good for some situations, but generally isn't worth the effort.

**N.B.** The objects are the same, but they are not singletons. A copy technically counts as a new instance, but does not call the constructor.

## Example

In [57]:
abstract class BasicCar implements Cloneable {
    protected String model;
    protected float price;
    
    public Object clone(){
        Object clone = null;
        
        try {
            clone = super.clone();
        } catch(CloneNotSupportedException exception){
            exception.printStackTrace();
        }
        
        return clone;
    }
}

In [58]:
class Ford extends BasicCar {
    public Ford(){
        model = "Ford";
        price = 234.55f;
    }
}

In [62]:
class CarCatalogue {
    private static Hashtable<String, BasicCar> carMap = 
                                                new Hashtable<String, BasicCar>();
    
    public static BasicCar getCarFromModel(String model){
        BasicCar cachedCar = carMap.get(model);
        return (BasicCar) cachedCar.clone();
    }
    
    public static void loadCache(){
        Ford ford = new Ford();
        carMap.put(ford.model, ford);
    }
}

In [63]:
CarCatalogue.loadCache();

In [65]:
BasicCar myFord = CarCatalogue.getCarFromModel("Ford");

In [66]:
System.out.println(myFord.price);

234.55


In [67]:
BasicCar my2ndFord = CarCatalogue.getCarFromModel("Ford");

In [68]:
System.out.println(my2ndFord.price);

234.55


The nicer way to implement this pattern...

In [78]:
class BasicCar {
    protected String model;
    protected float price;
    
    // If you want this to be a type 
    // can make the class abstract and remove
    // the implementations of the constructors
    // remember to make them abstract too though
    
    public BasicCar(String model, float price){
        this.model = model;
        this.price = price;
    }
    
    public BasicCar(BasicCar car){
        this.model = car.model;
        this.price = car.price;
    }
    
    public BasicCar copy(){
        return new BasicCar(this);
    }
}

In [79]:
class Ford extends BasicCar {
    public Ford(String model, float price){
        super(model, price);
    }
    
    public Ford(Ford car){
        super(car.model, car.price);
    }
    
    public Ford copy(){
        return new Ford(this);
    }
}

In [80]:
Ford myFord = new Ford("Ford Mustang", 23.99f);

In [81]:
Ford myFordTwo = myFord.copy();

In [82]:
System.out.println(myFord.model);
System.out.println(myFordTwo.model);

System.out.println(myFord);
System.out.println(myFordTwo);

Ford Mustang
Ford Mustang
REPL.$JShell$45F$Ford@664adc22
REPL.$JShell$45F$Ford@dc053cfa


Can also use polymorphism with the above if the abstract class was used. This is nicer as it has a reduced footprint codewise, but allows all of the same functionality as the regular pattern. It's also way less complicated.